In [ ]:
from apps.experiments.models import Experiment

import pandas as pd
from matplotlib import pyplot as plt

In [ ]:
experiment_jsons = []

for experiment in Experiment.objects.filter(fields=1):
    experiment_jsons.append({
        "method": experiment.method,
        "field": experiment.field_types,
        "count": experiment.count,
        "time": experiment.time,
    })


In [ ]:
df = pd.DataFrame(experiment_jsons)
df.sort_values(["method", "field", "count"], inplace=True)

In [ ]:
df

In [ ]:
field_average = {}

for field in df["field"].unique():
    field_average[field] = {}
    for method in [0, 1]:
        df2 = df[df.method == method]
        df2 = df2[df2.field == field]
        df2 = df2[["count", "time"]]
        
        field_average[field][method] = average = df2.groupby('count').mean()


In [ ]:
fig, axes = plt.subplots(3, 2, figsize=(6, 10))

order = [(0, 0), (0, 1), (1, 0), (1, 1), (2, 0)]

fig.delaxes(axes[2, 1])
fig.subplots_adjust(hspace=0.5, wspace=0.5)

for i, field in enumerate(df["field"].unique()):
    ax = axes[order[i]]
    
    ax.set_title(field)
    ax.set_xlabel("objects")
    ax.set_ylabel("seconds")
    ax.set_xlim(right=80000)
    ax.plot(field_average[field][Experiment.INBUILT].index, field_average[field][Experiment.INBUILT]["time"], label="inbuilt")
    ax.plot(field_average[field][Experiment.PACKAGE].index, field_average[field][Experiment.PACKAGE]["time"], label="package")
    ax.legend()

fig.savefig("single_field_comparison.pdf", tight_layout=True)
fig.show()